In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leningradskiy,RU,69.3833,178.4167,33.58,84,66,9.42,broken clouds
1,1,Westport,US,41.1415,-73.3579,65.53,75,0,6.91,clear sky
2,2,Elko,US,41.0002,-115.5012,53.64,39,0,3.00,clear sky
3,3,Hobart,AU,-42.8794,147.3294,41.05,75,75,11.50,broken clouds
4,4,Rosetta,EG,31.4044,30.4164,79.09,68,0,12.50,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 0
What is your desired maximum temperature for your trip? 100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leningradskiy,RU,69.3833,178.4167,33.58,84,66,9.42,broken clouds
1,1,Westport,US,41.1415,-73.3579,65.53,75,0,6.91,clear sky
2,2,Elko,US,41.0002,-115.5012,53.64,39,0,3.00,clear sky
3,3,Hobart,AU,-42.8794,147.3294,41.05,75,75,11.50,broken clouds
4,4,Rosetta,EG,31.4044,30.4164,79.09,68,0,12.50,clear sky
5,5,Baillif,GP,16.0202,-61.7463,78.30,80,16,11.86,few clouds
6,6,New Norfolk,AU,-42.7826,147.0587,40.71,82,34,1.99,scattered clouds
7,7,Provideniya,RU,64.3833,-173.3000,45.81,63,71,0.87,broken clouds
8,8,Qaanaaq,GL,77.4840,-69.3632,26.28,94,99,2.64,overcast clouds
9,9,Faanui,PF,-16.4833,-151.7500,79.99,74,51,14.41,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                8
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leningradskiy,RU,69.3833,178.4167,33.58,84,66,9.42,broken clouds
1,1,Westport,US,41.1415,-73.3579,65.53,75,0,6.91,clear sky
2,2,Elko,US,41.0002,-115.5012,53.64,39,0,3.00,clear sky
3,3,Hobart,AU,-42.8794,147.3294,41.05,75,75,11.50,broken clouds
4,4,Rosetta,EG,31.4044,30.4164,79.09,68,0,12.50,clear sky
...,...,...,...,...,...,...,...,...,...,...
689,689,Rosarito,MX,32.3333,-117.0333,64.42,86,38,2.82,scattered clouds
690,690,Rimbey,CA,52.6334,-114.2353,49.23,99,100,4.21,light rain
691,691,Boende,CD,-0.2167,20.8667,80.35,61,92,2.01,overcast clouds
692,692,Rocha,UY,-34.4833,-54.3333,44.51,85,19,3.13,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Leningradskiy,RU,33.58,broken clouds,69.3833,178.4167,
1,Westport,US,65.53,clear sky,41.1415,-73.3579,
2,Elko,US,53.64,clear sky,41.0002,-115.5012,
3,Hobart,AU,41.05,broken clouds,-42.8794,147.3294,
4,Rosetta,EG,79.09,clear sky,31.4044,30.4164,
5,Baillif,GP,78.30,few clouds,16.0202,-61.7463,
6,New Norfolk,AU,40.71,scattered clouds,-42.7826,147.0587,
7,Provideniya,RU,45.81,broken clouds,64.3833,-173.3000,
8,Qaanaaq,GL,26.28,overcast clouds,77.4840,-69.3632,
9,Faanui,PF,79.99,light rain,-16.4833,-151.7500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Sk

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Leningradskiy,RU,33.58,broken clouds,69.3833,178.4167,Gostinitsa
1,Westport,US,65.53,clear sky,41.1415,-73.3579,The Westport Inn
3,Hobart,AU,41.05,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
4,Rosetta,EG,79.09,clear sky,31.4044,30.4164,Green Valley
5,Baillif,GP,78.30,few clouds,16.0202,-61.7463,Hôtel Saint-Georges


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))